<a href="https://colab.research.google.com/github/callezenwaka/AWS-machine-learning/blob/main/capstone%20project%202/Deep_learning_for_single_cell_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Project Setup

Resources:

* https://towardsdatascience.com/integrative-analysis-of-single-cell-multi-omics-data-using-deep-learning-66a61a3448c5

In [6]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import requests as rq

In [ ]:
# download dataset
!wget https://github.com/callezenwaka/AWS-machine-learning/blob/main/capstone%20project%202/data.zip?raw=true && unzip -qq data.zip?raw=true
!rm -rf data.zip?raw=true

In [ ]:
# download helpe files
!wget https://github.com/callezenwaka/AWS-machine-learning/blob/main/capstone%20project%202/learn.zip?raw=true && unzip -qq learn.zip?raw=true
!rm -rf learn.zip?raw=true

In [ ]:
pip install umap-learn

In [11]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from pathlib import Path

from learn.dataset import TabularDataset
from learn.model import CiteAutoencoder
from learn.train import train_model, get_encodings

import umap
import plotly.express as px
import plotly.graph_objects as go

In [12]:
#
data_path = Path("./data/")
list(data_path.iterdir())

[PosixPath('data/metadata.csv.gz'),
 PosixPath('data/protein_scale.csv.gz'),
 PosixPath('data/rna_scale.csv.gz')]

In [13]:
#
rna = pd.read_csv(data_path/"rna_scale.csv.gz", index_col=0).T
rna = rna.reset_index(drop=True)
rna.head()

,IGKC,HBA2,HBB,HBA1,IGHA1,IGLC2,JCHAIN,HBM,IGHG1,IGHM,HBD,IGHG3,IGHG2,CA1,PTGDS,IGLC3,MZB1,AHSP,ELANE,HIST1H4C,IGLL1,PRTN3,CA2,MPO,DNTT,HBG2,AZU1,GYPA,PRDX2,HBG1,IGLL5,IGHGP,IGHA2,FAM178B,IGHG4,LYPD2,VPREB1,UBE2C,AL928768.3,APOC1,...,MAP7D2,ARPC1A,FAM46C,SVOPL,TRAV5,TMEM183A,RPL3,LINC00570,BRIX1,DNAJC2,TRBV21-1,LINC00114,RP11-556I14.1,LECT1,SLC16A8,CRIP1,SEL1L3,POLR3H,AMT,SERPINB1,RNF26,RP11-656D10.3,RBP4,MTIF2,INF2,RP11-693J15.5,NAE1,KDELR2,TCEAL2,SPATS2L,INSR,RAD23B,COMMD4,PPARA,PFDN6,PDSS1,BANF1,DDI2,DCAF6,HSPA5
0,-0.428204,0.183748,0.406723,-0.488968,1.248214,-0.614123,-0.324617,-0.181656,-0.200979,-0.414643,2.407124,-0.195287,-0.117689,-0.271599,-0.110263,-0.433555,-0.246179,-0.249721,-0.148742,-0.474248,1.514113,-0.12555,-0.180029,-0.226363,-0.08094,-0.046267,-0.167078,-0.131427,1.081326,-0.054701,-0.067024,-0.043884,-0.074466,-0.159364,-0.17881,-0.069823,-0.10839,2.493712,-0.137414,-0.160571,...,-0.038874,-0.317149,-0.177586,-0.075767,-0.03296,-0.333318,0.739594,-0.04443,-0.283894,2.660236,-0.017006,-0.014913,-0.014496,-0.016377,-0.015401,-0.471127,-0.241413,-0.178534,5.793597,2.086319,-0.183333,-0.021262,-0.021726,-0.159574,-0.14908,-0.154573,-0.324250,-0.445851,-0.038271,-0.117267,-0.216175,2.892558,-0.237605,-0.122832,-0.329040,-0.130763,-0.506981,3.164261,-0.197585,-0.44033
1,-1.047339,-0.601956,-0.985002,-0.488968,-0.552748,-0.614123,-0.324617,-0.181656,4.964708,-0.414643,-0.240882,-0.195287,-0.117689,-0.271599,-0.110263,-0.433555,-0.246179,-0.249721,-0.148742,-0.474248,-0.208137,-0.12555,6.346507,-0.226363,-0.08094,-0.046267,-0.167078,-0.131427,-0.523544,-0.054701,-0.067024,-0.043884,-0.074466,-0.159364,-0.17881,-0.069823,-0.10839,-0.175991,-0.137414,-0.160571,...,-0.038874,-0.317149,-0.177586,-0.075767,-0.03296,-0.333318,1.651260,-0.04443,-0.283894,-0.302578,-0.017006,-0.014913,-0.014496,-0.016377,-0.015401,-0.471127,-0.241413,-0.178534,-0.106664,-0.585515,-0.183333,-0.021262,-0.021726,-0.159574,-0.14908,-0.154573,-0.324250,-0.445851,-0.038271,-0.117267,-0.216175,-0.342985,-0.237605,-0.122832,-0.329040,-0.130763,-0.506981,-0.171434,-0.197585,-0.44033
2,-1.047339,-0.601956,1.534527,-0.488968,-0.552748,-0.614123,-0.324617,-0.181656,-0.200979,-0.414643,-0.240882,-0.195287,-0.117689,-0.271599,-0.110263,-0.433555,-0.246179,-0.249721,-0.148742,-0.474248,-0.208137,-0.12555,-0.180029,-0.226363,-0.08094,-0.046267,-0.167078,-0.131427,2.330472,-0.054701,-0.067024,-0.043884,-0.074466,-0.159364,-0.17881,-0.069823,-0.10839,-0.175991,-0.137414,-0.160571,...,-0.038874,-0.317149,-0.177586,-0.075767,-0.03296,-0.333318,0.393984,-0.04443,-0.283894,-0.302578,-0.017006,-0.014913,-0.014496,-0.016377,-0.015401,-0.471127,-0.241413,-0.178534,-0.106664,-0.585515,-0.183333,-0.021262,-0.021726,-0.159574,-0.14908,-0.154573,-0.324250,-0.445851,-0.038271,-0.117267,-0.216175,-0.342985,6.509144,-0.122832,-0.329040,-0.130763,-0.506981,-0.171434,-0.197585,-0.44033
3,0.080052,0.828744,0.614775,1.051572,-0.552748,0.713069,-0.324617,-0.181656,-0.200979,-0.414643,-0.240882,-0.195287,-0.117689,-0.271599,-0.110263,-0.433555,-0.246179,-0.249721,-0.148742,-0.474248,-0.208137,-0.12555,-0.180029,-0.226363,-0.08094,-0.046267,-0.167078,-0.131427,1.380851,-0.054701,-0.067024,-0.043884,-0.074466,-0.159364,-0.17881,-0.069823,-0.10839,-0.175991,-0.137414,-0.160571,...,-0.038874,3.054919,-0.177586,-0.075767,-0.03296,-0.333318,0.526239,-0.04443,-0.283894,-0.302578,-0.017006,-0.014913,-0.014496,-0.016377,-0.015401,-0.471127,-0.241413,-0.178534,-0.106664,-0.585515,-0.183333,-0.021262,-0.021726,-0.159574,-0.14908,-0.154573,2.931709,-0.445851,-0.038271,-0.117267,4.322459,-0.342985,4.264288,-0.122832,-0.329040,-0.130763,-0.506981,-0.171434,-0.197585,-0.44033
4,0.953832,-0.601956,0.794895,-0.488968,1.413335,-0.614123,-0.324617,-0.181656,-0.200979,-0.414643,-0.240882,-0.195287,-0.117689,-0.271599,-0.110263,-0.433555,-0.246179,-0.249721,-0.148742,1.555869,-0.208137,-0.12555,-0.180029,-0.226363,-0.08094,-0.046267,-0.167078,-0.131427,-0.523544,-0.054701,-0.067024,-0.043884,-0

In [14]:
#
pro = pd.read_csv(data_path/"protein_scale.csv.gz", index_col=0).T
pro = pro.reset_index(drop=True)
pro.head()

,CD11a,CD11c,CD123,CD127-IL7Ra,CD14,CD16,CD161,CD19,CD197-CCR7,CD25,CD27,CD278-ICOS,CD28,CD3,CD34,CD38,CD4,CD45RA,CD45RO,CD56,CD57,CD69,CD79b,CD8a,HLA.DR
0,-1.849174,-0.372667,-0.072177,-0.634849,-0.464785,0.207650,-0.005108,-0.361779,0.691453,0.492961,-0.812700,-0.211929,-0.244884,-0.845024,2.986220,1.046025,-0.578292,-0.685804,-0.618250,0.486534,1.084345,0.114363,-0.279477,-0.474619,0.998738
1,0.343613,-1.012818,-0.474588,1.776928,-0.672261,-0.152689,5.036511,-0.352564,-1.541172,-0.942280,-0.187590,0.815076,0.745067,0.897134,-0.652146,-0.665740,1.435716,-1.205478,0.359657,2.607521,-0.612902,-1.146097,0.245467,-0.762164,-0.838409
2,-0.829742,-0.847536,-0.675120,0.705449,-0.654620,-0.427962,-0.443269,-0.478648,-0.552103,-1.208599,1.355812,0.564809,1.467227,1.562876,-0.393136,0.010368,1.745735,0.426769,-1.072402,-0.223553,1.059687,-0.635409,-0.844739,-0.448139,-1.074838
3,0.157304,-0.601292,-0.558123,0.758353,-0.660516,-0.384782,0.052340,-0.413611,-0.338678,-0.098834,0.858286,1.845300,1.317319,1.137791,-0.565705,-0.202340,1.596914,-1.300920,2.089684,-0.637078,-0.166773,-0.541819,-0.646660,-0.663541,-0.589663
4,1.606986,1.762460,0.264567,-0.918552,2.004037,-0.497364,-0.392328,-0.327233,-0.539660,-0.375834,-1.080643,-0.885439,-0.986457,-0.789996,0.066885,1.264582,-0.308044,-1.246741,0.639829,-0.511057,0.300100,1.660178,-0.351614,-0.581021,0.766440


In [16]:
# 
nfeatures_rna = rna.shape[1]
nfeatures_pro = pro.shape[1]
print(nfeatures_rna, nfeatures_pro)

2000 25


In [17]:
# concat rna and pro
citeseq = pd.concat([rna, pro], axis=1)
citeseq.head()

,IGKC,HBA2,HBB,HBA1,IGHA1,IGLC2,JCHAIN,HBM,IGHG1,IGHM,HBD,IGHG3,IGHG2,CA1,PTGDS,IGLC3,MZB1,AHSP,ELANE,HIST1H4C,IGLL1,PRTN3,CA2,MPO,DNTT,HBG2,AZU1,GYPA,PRDX2,HBG1,IGLL5,IGHGP,IGHA2,FAM178B,IGHG4,LYPD2,VPREB1,UBE2C,AL928768.3,APOC1,...,RP11-693J15.5,NAE1,KDELR2,TCEAL2,SPATS2L,INSR,RAD23B,COMMD4,PPARA,PFDN6,PDSS1,BANF1,DDI2,DCAF6,HSPA5,CD11a,CD11c,CD123,CD127-IL7Ra,CD14,CD16,CD161,CD19,CD197-CCR7,CD25,CD27,CD278-ICOS,CD28,CD3,CD34,CD38,CD4,CD45RA,CD45RO,CD56,CD57,CD69,CD79b,CD8a,HLA.DR
0,-0.428204,0.183748,0.406723,-0.488968,1.248214,-0.614123,-0.324617,-0.181656,-0.200979,-0.414643,2.407124,-0.195287,-0.117689,-0.271599,-0.110263,-0.433555,-0.246179,-0.249721,-0.148742,-0.474248,1.514113,-0.12555,-0.180029,-0.226363,-0.08094,-0.046267,-0.167078,-0.131427,1.081326,-0.054701,-0.067024,-0.043884,-0.074466,-0.159364,-0.17881,-0.069823,-0.10839,2.493712,-0.137414,-0.160571,...,-0.154573,-0.324250,-0.445851,-0.038271,-0.117267,-0.216175,2.892558,-0.237605,-0.122832,-0.329040,-0.130763,-0.506981,3.164261,-0.197585,-0.44033,-1.849174,-0.372667,-0.072177,-0.634849,-0.464785,0.207650,-0.005108,-0.361779,0.691453,0.492961,-0.812700,-0.211929,-0.244884,-0.845024,2.986220,1.046025,-0.578292,-0.685804,-0.618250,0.486534,1.084345,0.114363,-0.279477,-0.474619,0.998738
1,-1.047339,-0.601956,-0.985002,-0.488968,-0.552748,-0.614123,-0.324617,-0.181656,4.964708,-0.414643,-0.240882,-0.195287,-0.117689,-0.271599,-0.110263,-0.433555,-0.246179,-0.249721,-0.148742,-0.474248,-0.208137,-0.12555,6.346507,-0.226363,-0.08094,-0.046267,-0.167078,-0.131427,-0.523544,-0.054701,-0.067024,-0.043884,-0.074466,-0.159364,-0.17881,-0.069823,-0.10839,-0.175991,-0.137414,-0.160571,...,-0.154573,-0.324250,-0.445851,-0.038271,-0.117267,-0.216175,-0.342985,-0.237605,-0.122832,-0.329040,-0.130763,-0.506981,-0.171434,-0.197585,-0.44033,0.343613,-1.012818,-0.474588,1.776928,-0.672261,-0.152689,5.036511,-0.352564,-1.541172,-0.942280,-0.187590,0.815076,0.745067,0.897134,-0.652146,-0.665740,1.435716,-1.205478,0.359657,2.607521,-0.612902,-1.146097,0.245467,-0.762164,-0.838409
2,-1.047339,-0.601956,1.534527,-0.488968,-0.552748,-0.614123,-0.324617,-0.181656,-0.200979,-0.414643,-0.240882,-0.195287,-0.117689,-0.271599,-0.110263,-0.433555,-0.246179,-0.249721,-0.148742,-0.474248,-0.208137,-0.12555,-0.180029,-0.226363,-0.08094,-0.046267,-0.167078,-0.131427,2.330472,-0.054701,-0.067024,-0.043884,-0.074466,-0.159364,-0.17881,-0.069823,-0.10839,-0.175991,-0.137414,-0.160571,...,-0.154573,-0.324250,-0.445851,-0.038271,-0.117267,-0.216175,-0.342985,6.509144,-0.122832,-0.329040,-0.130763,-0.506981,-0.171434,-0.197585,-0.44033,-0.829742,-0.847536,-0.675120,0.705449,-0.654620,-0.427962,-0.443269,-0.478648,-0.552103,-1.208599,1.355812,0.564809,1.467227,1.562876,-0.393136,0.010368,1.745735,0.426769,-1.072402,-0.223553,1.059687,-0.635409,-0.844739,-0.448139,-1.074838
3,0.080052,0.828744,0.614775,1.051572,-0.552748,0.713069,-0.324617,-0.181656,-0.200979,-0.414643,-0.240882,-0.195287,-0.117689,-0.271599,-0.110263,-0.433555,-0.246179,-0.249721,-0.148742,-0.474248,-0.208137,-0.12555,-0.180029,-0.226363,-0.08094,-0.046267,-0.167078,-0.131427,1.380851,-0.054701,-0.067024,-0.043884,-0.074466,-0.159364,-0.17881,-0.069823,-0.10839,-0.175991,-0.137414,-0.160571,...,-0.154573,2.931709,-0.445851,-0.038271,-0.117267,4.322459,-0.342985,4.264288,-0.122832,-0.329040,-0.130763,-0.506981,-0.171434,-0.197585,-0.44033,0.157304,-0.601292,-0.558123,0.758353,-0.660516,-0.384782,0.052340,-0.413611,-0.338678,-0.098834,0.858286,1.845300,1.317319,1.137791,-0.565705,-0.202340,1.596914,-1.300920,2.089684,-0.637078,-0.166773,-0.541819,-0.646660,-0.663541,-0.589663
4,0.953832,-0.601956,0.794895,-0.488968,1.413335,-0.614123,-0.324617,-0.181656,-0.200979,-0.414643,-0.240882,-0.195287,-0.117689,-0.271599,-0.110263,-0.433555,-0.246179,-0.249721,-0.148742,1.555869,-0.208137,-0.12555,-0.180029,-0.226363,-0.08094,-0.046267,-0.167078,-0.131427,-0.523544,-0.054701,-0.067024,-0.043884,-0.074466,-0.159364,-0.17881,-0.069823,-0.10839,-0.1

In [21]:
#
train, valid = train_test_split(citeseq.to_numpy(dtype=np.float32), test_size=0.1, random_state=0)
print(f"train shape: {train.shape}\nvalid shape: {valid.shape}")

train shape: (27604, 2025)
valid shape: (3068, 2025)


In [22]:
train[0]

array([ 1.2470618 , -0.6019564 , -0.9850023 , ..., -0.13890247,
       -0.5083658 , -1.1323029 ], dtype=float32)

In [23]:
train_ds = TabularDataset(train)
valid_ds = TabularDataset(valid)

In [24]:
train_dl = DataLoader(train_ds, batch_size=64, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=64, shuffle=False)

In [25]:
x, y = next(iter(train_dl))
x.shape, y.shape

(torch.Size([64, 2025]), torch.Size([64, 2025]))

In [26]:
model = CiteAutoencoder(nfeatures_rna, nfeatures_pro, hidden_rna=85, hidden_pro=15, z_dim=20)
model

CiteAutoencoder(
  (encoder): Encoder(
    (encoder_rna): LinBnDrop(
      (0): Linear(in_features=2000, out_features=85, bias=False)
      (1): LeakyReLU(negative_slope=0.01)
      (2): BatchNorm1d(85, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.1, inplace=False)
    )
    (encoder_protein): LinBnDrop(
      (0): Linear(in_features=25, out_features=15, bias=False)
      (1): LeakyReLU(negative_slope=0.01)
      (2): BatchNorm1d(15, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): Dropout(p=0.1, inplace=False)
    )
    (encoder): LinBnDrop(
      (0): Linear(in_features=100, out_features=20, bias=False)
      (1): LeakyReLU(negative_slope=0.01)
      (2): BatchNorm1d(20, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (decoder): Decoder(
    (decoder): Sequential(
      (0): LinBnDrop(
        (0): Linear(in_features=20, out_features=100, bias=False)
        (1): LeakyReLU(negative_slope=0

In [27]:
#
model(x).shape

torch.Size([64, 2025])

In [29]:
#
lr = 1e-2
epochs = 50
model, losses = train_model(model, train_dl, valid_dl, lr=lr, epochs=epochs)

 20%|██        | 10/50 [00:32<02:11,  3.28s/it]

Epoch 10: train loss 0.6292299920796415; valid loss 0.6195559693595111


 40%|████      | 20/50 [01:05<01:38,  3.27s/it]

Epoch 20: train loss 0.6240642445965999; valid loss 0.6136219873732973


 60%|██████    | 30/50 [01:38<01:05,  3.27s/it]

Epoch 30: train loss 0.6189471758408679; valid loss 0.6096530264483904


 80%|████████  | 40/50 [02:10<00:32,  3.26s/it]

Epoch 40: train loss 0.613709561229875; valid loss 0.6069777179727964


100%|██████████| 50/50 [02:43<00:00,  3.27s/it]

Epoch 50: train loss 0.6110849949505481; valid loss 0.6060085505674528


In [30]:
# 
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(1, epochs+1, 1), y=losses['train'], mode='lines', name='train'))
fig.add_trace(go.Scatter(x=np.arange(1, epochs+1, 1), y=losses['valid'], mode='lines', name='valid'))
fig.show()

In [31]:
#
test_ds = TabularDataset(citeseq.to_numpy(dtype=np.float32))
test_dl = DataLoader(test_ds, batch_size=64, shuffle=False)

encodings = get_encodings(model, test_dl)
encodings = encodings.cpu().numpy()
encodings.shape

(30672, 20)

In [33]:
# annotations
metadata = pd.read_csv(data_path/"metadata.csv.gz", index_col=0)
metadata.head()

,orig.ident,nCount_RNA,nFeature_RNA,nCount_ADT,nFeature_ADT,lane,donor,celltype.l1,celltype.l2,RNA.weight,ADT.weight,wsnn_res.2,seurat_clusters
a_AAACCTGAGCTTATCG-1,bmcite,7546,2136,1350,25,HumanHTO4,batch1,Progenitor cells,Prog_RBC,0.487299,0.512701,19,19
a_AAACCTGAGGTGGGTT-1,bmcite,1029,437,2970,25,HumanHTO1,batch1,T cell,gdT,0.245543,0.754457,10,10
a_AAACCTGAGTACATGA-1,bmcite,1111,429,2474,23,HumanHTO5,batch1,T cell,CD4 Naive,0.501680,0.498320,1,1
a_AAACCTGCAAACCTAC-1,bmcite,2741,851,4799,25,HumanHTO3,batch1,T cell,CD4 Memory,0.431308,0.568692,4,4
a_AAACCTGCAAGGTGTG-1,bmcite,2099,843,5434,25,HumanHTO2,batch1,Mono/DC,CD14 Mono,0.572097,0.427903,2,2


In [34]:
# separate CD4 and CD8 in l1
metadata["celltype.l1.5"] = metadata["celltype.l1"].values
metadata.loc[metadata["celltype.l2"].str.startswith("CD4"), "celltype.l1.5"] = "CD4 T"
metadata.loc[metadata["celltype.l2"].str.startswith("CD8"), "celltype.l1.5"] = "CD8 T"
metadata.loc[metadata["celltype.l2"]=="Treg", "celltype.l1.5"] = "CD4 T"
metadata.loc[metadata["celltype.l2"]=="MAIT", "celltype.l1.5"] = "MAIT"
metadata.loc[metadata["celltype.l2"]=="gdT", "celltype.l1.5"] = "gdT"

In [35]:
#
embedding = umap.UMAP(random_state=0).fit_transform(encodings)

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning:

The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.



In [36]:
# 
plot_df = metadata.copy()
plot_df["UMAP1"] = embedding[:, 0]
plot_df["UMAP2"] = embedding[:, 1]

In [37]:
#
fig = px.scatter(plot_df, x="UMAP1", y="UMAP2", color="celltype.l1.5")
fig.update_traces(marker=dict(size=4))
fig.show()